# Assignment 3 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Personal Details:

In [1]:
# Details Student 1:
name = "itay marlinsky"
id = 208634691
mail = 'itaymerel1212@gmail.com'
# Details Student 2:

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [2]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [3]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [4]:
# word net installation:

# unmark if you want to use and need to install
!pip install wn
!python -m wn download omw-he:1.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 11.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Download [##############################] (315276/315276 bytes) Complete
Read [##############################] (29690/29690) 
Added omw-he:1.4 (Hebrew Wordnet)



In [5]:
# word net import:

# unmark if you want to use:
import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [6]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
# !pip install hebrew_tokenizer

In [7]:
# Hebrew tokenizer import:

# unmark if you want to use:
# import hebrew_tokenizer as ht

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [8]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [26]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",0
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,0
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,1
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",0
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,1
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,1
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,0
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,1


(753, 2)

In [25]:
df_test.head(10)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...
3,3,"רגע הגיוס לצבא היה הרגע הכי משמעותי עבורי, אני..."
4,4,אני הגעתי לברזיל ישר מקולומביה וגם אני עשיתי ע...
5,5,בפעם האחרונה שהייתי מחוץ לארץ ישראל הייתי באפר...
6,6,בשנת 2018 קיבלתי החלטה שאני מתחיל ללמוד לתואר ...
7,7,"בנובמבר האחרון הייתי עם חברים בטיול ים אל ים, ..."
8,8,"לפני מספר חודשים, ביום שמש בהיר, קמתי בבוקר למ..."
9,9,"אני לא בן אדם שנוטה לשתף בחייו האישיים, אבל או..."


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

In [11]:
# YOUR CODE HERE
from sklearn.linear_model import LogisticRegression
gender_map = {'m': 0, 'f': 1}
df_train['gender'] = df_train['gender'].map(gender_map)



In [35]:


vec = TfidfVectorizer( encoding='utf-8', decode_error='strict', strip_accents=None, lowercase=True, 
                       analyzer='word',  max_features=None 
                      , norm='l2')


sample1 = df_test['story'][0:5]
sample2 = df_test['story'][-6:-1]


# Transform the train and test data
X_train = vec.fit_transform(df_train['story'])
X_test = vec.transform(df_test['story'][5:-5])

#normalization
X_train = preprocessing.normalize(X_train)
X_test = preprocessing.normalize(X_test)
# Encoding gender labels
y_train = df_train['gender']


#load model
clf = MultinomialNB()
model = LogisticRegression()
decisionTree = tree.DecisionTreeClassifier(max_depth=20, min_samples_split=11)

#cross_validation
df_predicted = cross_val_score(decisionTree, X_train, y_train, cv=10,scoring='f1_macro')

average_f1 = df_predicted.mean()

print("Average F1 Score:", average_f1)

for x in df_predicted:
   print("test result: " ,x)




Average F1 Score: 0.5962458011824185
test result:  0.5935828877005348
test result:  0.5745134630765129
test result:  0.6059907834101382
test result:  0.5714285714285714
test result:  0.5608899297423887
test result:  0.5833333333333334
test result:  0.5730550284629982
test result:  0.49599999999999994
test result:  0.6950393060449988
test result:  0.7086247086247086


In [39]:
df_predicted = pd.DataFrame()
df_predicted['test_example_id'] = df_test['test_example_id'][5:-5]
decisionTree = tree.DecisionTreeClassifier(max_depth=20, min_samples_split=11)
decisionTree = decisionTree.fit(X_train,y_train)
df_predicted['predicted_category'] = decisionTree.predict(X_test)
print(df_predicted)

     test_example_id  predicted_category
5                  5                   0
6                  6                   0
7                  7                   0
8                  8                   0
9                  9                   0
..               ...                 ...
313              313                   0
314              314                   1
315              315                   0
316              316                   0
317              317                   1

[313 rows x 2 columns]


In [38]:
X1 = vec.transform(sample1) 
X2 = vec.transform(sample2)

decisionTree = tree.DecisionTreeClassifier(max_depth=20, min_samples_split=11)
decisionTree = decisionTree.fit(X_train,y_train)
# Predict the gender of the first sample sentence.
predicted_gender = decisionTree.predict(X1)
print("The predicted gender of the first sample sentence is:", predicted_gender)

predicted_gender = decisionTree.predict(X2)
print("The predicted gender of the second sample sentence is:", predicted_gender)
    





The predicted gender of the first sample sentence is: [1 0 0 0 1]
The predicted gender of the second sample sentence is: [1 0 0 1 0]


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [40]:
df_predicted.to_csv('classification_results.csv',index=False)